# Tensor是什么
torch中的tensor类似numpy的ndarrays, 是最基本的数据结构，用于存储和操作多维数组。

In [ ]:
import torch
import numpy as np


___

### 创建tensor

In [ ]:
# 列表到张量
tensor_from_list = torch.tensor([1,2,30])
# ndarray 搭配张量
np_array = np.array([4,5,6])
tensor_from_numpy = torch.tensor(np_array)
print(tensor_from_numpy)

In [ ]:
# 全零
zero_tensor = torch.zeros((3, 4))
print(zero_tensor)
# 全1
ones_tensor = torch.ones((2,2))
print(ones_tensor)
# 指定范围张量
range_tensor = torch.arange(0, 10, 2)
print(range_tensor)
# 均匀分布的tensor
mean_tensor = torch.rand((3,3))
print(mean_tensor)
# 正态分布tensor
normal_tensor = torch.randn((3,3))
print(normal_tensor)

In [ ]:
# 未初始化的tensor
empty_tensor = torch.empty((2,2))

# 常见与某个tensor形状相同的tensor
same_shape_tensor = torch.ones_like(normal_tensor)
print(empty_tensor)
print(same_shape_tensor)


___

## 张量的基本操作

In [ ]:
# 张量的尺寸
# tensor_from_list = torch.tensor([1,2,30])
shape_tensor = tensor_from_list.shape
print(shape_tensor)

# 索引
element = tensor_from_list[0]
print(element)

# 切片
sliced_tensor = tensor_from_list[:2]
print(sliced_tensor)

In [ ]:
# 改变张量的形状
reshaped_tensor = tensor_from_list.view(1,3)
# 改成1，3相当于从一维张量转为了二维张量
print(reshaped_tensor)
# 张量转置
transposed_tensor = tensor_from_list.t()
print(transposed_tensor)

**数学计算**

In [ ]:
tensor_o = torch.ones(1,3)
tensor_rand = torch.randn(1,3)
print(tensor_o, tensor_rand)

#加法操作
tensor_sum = tensor_o + tensor_rand
print("sum:", tensor_sum)

# 乘法操作: 注意matmul是典型的矩阵乘法
product_tensor = torch.matmul(tensor_sum, tensor_o.t())
print(product_tensor)
product_tensor_2 = torch.matmul(tensor_sum.t(), tensor_o)
print(product_tensor_2)

# 广播计算
tensor_o_broad = tensor_o * 3
print(tensor_o_broad)
tensor_o_broad_2 = tensor_o_broad + 2
print(tensor_o_broad_2)

## 自动求导


# 自动求导
pytorch自动求导的核心是torch.autograd模块，它为对张量的所有操作提供了自动求导服务，其核心部分包含:
+ tensor
+ 计算图
+ 梯度计算
+ 梯度累计
+ 离散跟踪

如果张量的requires_grad属性被设置为True，那么对它的每一步操作都将被跟踪。当结束计算后，可以调用**backward()**方法，来自动计算所有参数的梯度，注意这个梯度是**累加的**

In [ ]:
x = torch.randn(3, 3, requires_grad = True)
print(x.grad_fn)
# 由于x是用户创建的叶子节点，因此没有跟踪计算图

In [ ]:
x = torch.ones(2,2, requires_grad= True)
print(x)

In [ ]:
# 对张量进行一次计算
y = x ** 2
print(y)
print(y.grad_fn)
# more
z = y * y * 3
out = z.mean()
print(z, out)

由于y是对x进行操作的结果，因此这个操作会被跟踪，y的grad_fn属性为grad_fn=<PowBackward0>, 字面上看是幂计算反向传播。

In [ ]:
# requires_grad属性默认是false
a = torch.randn(3,2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
# .requires_grad_()方法可以原地修改张量的requires_grad属性
a.requires_grad_(True)
b = (a * a).sum()
print(b.grad_fn)

___

## 梯度

In [ ]:
# 对out反向传播
out.backward()

In [ ]:
# 看看d(out)/dx
print(x)
print(x.grad)

In [ ]:
# 雅可比向量积的例子:
j = torch.randn(3, requires_grad=True)
print(j)

k = j * 2
m = 0
while k.data.norm() < 1000:
    k = k * 2
    m = m + 1
print(k)
print(m)
# 此时k已经不再是标量，autograd不能直接计算梯度，因此只可以求雅可比向量积。
v = torch.tensor([0.1, 1.0, 0.0001],dtype=torch.float)
k.backward(v)
print(j.grad)

注意，auto_grad只能直接求**标量**结果的反向传播结果。如果结果是向量的话，对所有输入求导就会得到一个**雅可比矩阵**，而不是该输入的梯度值。因此，如果最终结果是向量的话，backward()需要引入一个与输出同维度的权重向量。

___

***梯度的累加机制***

In [ ]:
# 如果不手动对参数的梯度清零的话，其梯度会一直累加
out2 = x.sum()
out2.backward()
print(x.grad)
out3 = x.sum()
x.grad.data.zero_()
out3.backward()
print(x.grad)

___

## 离散追踪
推理或者验证模型的时候，我们不希望数值计算过程被追踪，因为那样会消耗额外的内存与算力。此时使用torch.no_grad()来临时将tensor的requires_grad属性设置为False。

In [ ]:
print(j.requires_grad)
print((j ** 2).requires_grad)

with torch.no_grad():
    print((j ** 2).requires_grad)

___
tip:如果我们希望改变一个tensor的值，却不想该百年它的autograd记录，那么就直接对tensor.data属性进行操作：

In [ ]:
x = torch.ones(1, requires_grad=True)

print(x.data)  # 其实还是一个tensor
print(x.data.requires_grad)

y = 2 * x
x.data *= 100 # 只改变了值，不会记录在计算图，所以不会影响梯度传播
y.backward()
print(x)
print(x.grad)  # 发现导数为2， 与100无关

___

## grad_fn属性详解

当我们对张量进行运算的时候，得到的张量的grad_fn会自动记录该操作，**并指向一个Function对象**，该对象复制前向传播和反向传播。tensor与Function共同组成一个无环有向的计算图。



# 多gpu并行计算

### 什么是CUDA?
CUDA是英伟达GPU的并行运算框架。对GPU的编程是使用CUDA语言完成的。然而pytorch的cuda意思是我们即将使用GPU来处理数据和模型。

当我们希望把模型或者数据从cpu迁移到gpu时，就需要使用.cuda()方法（默认0号gpu）。
tips：
+ pytorch目前不支持amd的opengl接口
+ 避免频繁地将数据在cpu和gpu之间切换
+ 进行简单操作时，使用cpu

——————

如何设置默认显卡？两种方法


In [ ]:
 #设置在文件最开始部分
import os
os.environ["CUDA_VISIBLE_DEVICE"] = "2" # 设置默认的显卡
# 或者
CUDA_VISBLE_DEVICE=0,1 python train.py # 使用0，1两块GPU

## 常见的并行训练方法：

**1.mambaout,模型拆分**  
![模型拆分图解](https://datawhalechina.github.io/thorough-pytorch/_images/model_parllel.png)  
该方法难在gpu之间的通信

**2.同一层任务分不到不同数据中**
这种方法将同一层模型做拆分，不太理解  

**3.不同数据拆分到不同设备(数据并行方式, data parallelism)**  
![](https://datawhalechina.github.io/thorough-pytorch/_images/data_parllel.png)  
该方法的逻辑是将相同的模型复制到各个显卡中，然后将数据切分，让各个显卡训练数据的一部分，最后进行汇总反向传播。这样可以解决通信问题。

___


### 多卡训练 
pytorch提供了data parallel(DP)方式和DistributedDataParallel(DDP)两种多卡训练方式

***使用CUDA加速训练***  
**单卡训练**  
非常简单，只要将模型和数据.cuda()就行了  
首先要手动指定对程序可见的gpu

**多卡DP**

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2"

In [ ]:
model = Net()
model.cuda() # 模型显示转移到CUDA上

if torch.cuda.device_count() > 1: # 含有多张GPU的卡
	model = nn.DataParallel(model) # 单机多卡DP训练

DP的抽象过程，数据被分为多个子集后，分别在各个gpu计算梯度，最后把梯度汇总到一个主gpu上进行参数的更新，这样会使得主gpu的工作负担明显大于其他gpu，成为性能瓶颈。此外，DP只适合单机模式。**注意DP是单进程多线程**

问题：既然python解释器有GIL锁，为什么限制不了DP的加速？
因为pytorch数据运行于gpu上，由CUDA和其他GPU加速库管理，不受python解释器管理。

**多机多卡DDP**  
DDP是一种多卡多进程方式，每个进程独立运行在一个gpu上，每个gpu独立更新参数，使用高效的通信机制进行梯度信息同步。

## GIL锁对于DP与DDP的影响？
由于模型训练大部分内容是在gpu上运行的，因此对于DP，GIL锁对工作的大部分内容几乎没有影响。
但是对于**运行于cpu的数据预处理加载和IO操作，还有数据后处理和日志记录等**，可能成为性能瓶颈。
但是对于多进程的DDP，由于各个进程都有独立的python解释器，实现了真正的并行运行，因此不受GIL影响。

这对于严重依赖 Python runtime 的 models 而言，比如说包含 RNN 层或大量小组件的 models 而言，这尤为重要。什么是python runtime？？？

# 深度学习训练的整体流程
+ 数据预处理： 数据格式统一、异常数据清除、数据变换
+ 划分训练集、验证集、测试集（可以使用sklearn自带的test_train_split函数）
+ 模型选择、损失函数、优化方法
+ 超参数设置
+ 使用模型拟合训练数据集，在验证集、测试机上评估模型表现



## 包的导入

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optimizer

In [ ]:
# 显式指定gpu
os.environ['CUDA_VISIBLE_DEVICES'] = '0, 1' #指明gpu为01
device = torch.device("cuda:1" if torch.cuda.is_available else cpu)
print(device)

In [ ]:
# 设置一些超参数
batch_size = 16

lr = 1e-4

epoch = 100
# 当然我们可以选择把超参数存储在yaml里面

## 数据读入
+ dataset定义数据格式和数据变形形式
+ dataloader以迭代的方式，向模型输入批次数据

___

## dataset
**使用torch自建的dataset ImageFolder**

In [ ]:
# 以cifar10数据集构建Dataset类的方式
import torch
from torchvision import datasets
# 这里使用了PyTorch自带的ImageFolder类的用于读取按一定结构存储的图片数据（path对应图片存放的目录，目录下包含若干子目录，每个子目录对应属于同一个类的图片）
train_data = datasets.ImageFolder(train_path, transfor=data_transform)
val_data = datasets.ImageFolder(val_path, transform=data_transform)

**使用自定义的dataset**

In [ ]:
import os
import pandas as pd
from torchvision.io import read_image

class MyDataset(Dataset):
    # 注意实现Dataset父类
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self, idx):
        """
        image1.jpg, 0
        image2.jpg, 1
        ......
        image9.jpg, 9
        
        """
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        # 数据加载
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

构建完成dataset以后，就可以使用dataloader批次读入数据了。

In [ ]:
import torch.utils
from torch.utils.data import DataLoader

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=4, shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, num_workers=4, shuffle=False)
# 参数解析
# num_workers, cpu读取数据的进程数
# drop_last, 对于最后达不到batch_size数量的样本，是否丢弃？

___

## 模型构建
**Module 类**是 torch.nn 模块里提供的一个模型构造类，是所有神经网络模块的基类，我们可以继承它来定义我们想要的模型。



In [7]:
# 使用nn.Module构建多层感知机
import torch
from torch import nn

class MLP(nn.Module):
    # 在__init__中构建神经网络
    def __init__(self, **kwargs):
    # 调用MLP父类Block的构造函数来进行必要的初始化。这样在构造实例时还可以指定其他函数
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Linear(784,256)
        self.act = nn.ReLU()
        self.output = nn.Linear(256, 10)
    
    # 定义模型的前向计算
    def forward(self, x):
        o = self.act(self.hidden(x))
        return self.output(o)

In [8]:
# 随机设置一个输入
x = torch.randn(2, 784) # 第一维是batch
net = MLP()
print(net)
net(x)  # 前向计算

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[ 0.1897, -0.2245, -0.0114, -0.3305,  0.2267, -0.1157,  0.2367, -0.0760,
          0.0160,  0.0921],
        [-0.0816, -0.0649,  0.1001, -0.0160,  0.1412, -0.2122, -0.3227, -0.3230,
         -0.2738, -0.1120]], grad_fn=<AddmmBackward0>)

### 深度学习有各种各样的层, 本节学习使用torch构建自定义层

In [9]:
# 不含参数的层
import torch
from torch import nn

class MyLayer(nn.Module):
    def __init__(self, **kwargs):
        super(MyLayer, self).__init__(**kwargs)
    def forward(self, x):
        return x - x.mean()  

In [10]:
# 自定义参数的层，注意Parameter是Tensor的子类型，会被自动添加到模型的参数列表之中,使之可以训练
# 这样的话，似乎就可以实现模型手撕了
class MyListDense(nn.Module):
    def __init__(self):
        super(MyListDense, self).__init__()
        self.params = nn.ParameterList([nn.Parameter(torch.randn(4, 4)) for i in range(3)])
        self.params.append(nn.Parameter(torch.randn(4, 1)))

    def forward(self, x):
        for i in range(len(self.params)):
            # mm，仅仅可以对付二维矩阵乘法
            x = torch.mm(x, self.params[i])
        return x
net = MyListDense()
print(net)

class MyDictDense(nn.Module):
    def __init__(self):
        super(MyDictDense, self).__init__()
        self.params = nn.ParameterDict({
                'linear1': nn.Parameter(torch.randn(4, 4)),
                'linear2': nn.Parameter(torch.randn(4, 1))
        })
        self.params.update({'linear3': nn.Parameter(torch.randn(4, 2))}) # 新增

    def forward(self, x, choice='linear1'):
        return torch.mm(x, self.params[choice])

net = MyDictDense()
print(net)

MyListDense(
  (params): ParameterList(
      (0): Parameter containing: [torch.float32 of size 4x4]
      (1): Parameter containing: [torch.float32 of size 4x4]
      (2): Parameter containing: [torch.float32 of size 4x4]
      (3): Parameter containing: [torch.float32 of size 4x1]
  )
)
MyDictDense(
  (params): ParameterDict(
      (linear1): Parameter containing: [torch.FloatTensor of size 4x4]
      (linear2): Parameter containing: [torch.FloatTensor of size 4x1]
      (linear3): Parameter containing: [torch.FloatTensor of size 4x2]
  )
)


In [11]:
# 手撕二维卷积！！！

import torch
from torch import nn

def corr2d(X, K):
    h, w = K.shape
    print(h,w)
    X, K = X.float(), K.float()
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i: i + h, j: j + w] * K).sum()
    return Y

class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super(Conv2D, self).__init()
        # super 方法用于为子类调用父类方法，type参数
        # 传入类型， obj传入实例，在类定义中, obj传入
        # self
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.randn(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

In [13]:
# 使用padding进行same卷积
import torch
from torch import nn

def comp_conv2d(conv2d, X):
    X = X.view((1,1) + X.shape) # 为X加入批次和通道维度
    Y = conv2d(X)
    return Y.view(Y.shape[2:]) # 滤除批次和通道维度

# 对输入左右padding两行
conv2d = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, padding= 1)

X = torch.rand(8,8)
# 注意卷积之后前后变换公式: y = (x - h + 2 * p)/s + 1, 注意向下取整
print(comp_conv2d(conv2d, X).shape)

# 使用高宽不一致的卷积核
conv2d = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(5, 3), padding=(2, 1))
print(comp_conv2d(conv2d, X).shape)

# 引入stride
conv2d = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(3, 5), padding= (0, 1), stride = (3, 4))
print(comp_conv2d(conv2d, X).shape)



torch.Size([8, 8])
torch.Size([8, 8])
torch.Size([2, 2])


In [17]:
# 手撕池化层
import torch
from torch import nn

def pool2d(X, pool_size, mode = 'max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i: i + p_h, j: j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i: i + p_h, j: j + p_w].mean()
    return Y

X = torch.tensor([[0,1,2],[4,5,6],[7,8,9]], dtype = torch.float)
print(pool2d(X, (2,2)))
print(pool2d(X,(2,2), "avg"))

tensor([[5., 6.],
        [8., 9.]])
tensor([[2.5000, 3.5000],
        [6.0000, 7.0000]])


___

# 模型初始化

### 为什么参数初始化不能全部为0  
当网络中所有权重都初始化为0时，对于任何隐藏层，所有神经元在前向传播和反向传播过程中的行为都会完全相同。这意味着每一层的每个神经元将计算出相同的输出，并且在反向传播中也会接收到相同的梯度。结果是，即使网络的参数被更新，每个权重也会保持相同，这使得隐藏层的多个神经元没有区分开来，因此无法学习到多样化的特征或模式。  
此外，对于一些过零的激活函数（如Sigmoid或Tanh），全部初始化为0也会导致梯度消失问题，当激活函数的输入接近0的时候，其反向传播获得的梯度也会接近0。